In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import thu vien
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import string
import os
from sklearn.metrics import f1_score, accuracy_score
from math import log
#Download the stopwords package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Tao ham cleanEmail de tach cac ki tu ngan cach, cac tu khong lien quan, gay nhieu du lieu
def cleanEmail(email):
  mylist =[]

  for char in email:
    if char not in string.punctuation:
      mylist.append(char)
  mylist = ''.join(mylist)

  cleanText = ""
  for word in mylist.split():
    if word.lower() not in stopwords.words('english') :
        cleanText = cleanText + " " + word
  return cleanText

# Ham tao dataframe cua tap Train va tap Test
def ListEmail(dir, file_list):
  myString = []
  for file in file_list:
    with open(dir + '/' + file) as f:
      email = cleanEmail(f.read())
    myString.append(email)
  return myString

# Ham tao DataFrame tap Train va test
def makeDataFrame(dirNotSpam, dirSpam):
  FileListNotSpam = os.listdir(dirNotSpam)
  FileListSpam = os.listdir(dirSpam)
  Emails1 = ListEmail(dirNotSpam, FileListNotSpam)
  dfns = pd.DataFrame({
      'Email': Emails1,
      'Label':  np.zeros(len(Emails1), dtype=int)
  }) # 0 la khong spam
  Emails2 = ListEmail(dirSpam, FileListSpam)
  dfs = pd.DataFrame({
      'Email': Emails2,
      'Label':  np.ones(len(Emails2), dtype=int)
  }) # 1 la spam
  df = pd.concat([dfns, dfs], ignore_index=True)
  return df

In [ ]:
dirTrainNotSpam = '/content/drive/MyDrive/Colab Notebooks/SamSungTest2/TrainData/notspam'
dirTrainSpam = '/content/drive/MyDrive/Colab Notebooks/SamSungTest2/TrainData/spam'
dirTestNotSpam = '/content/drive/MyDrive/Colab Notebooks/SamSungTest2/TestData_withlabel/notspamTest'
dirTestSpam = '/content/drive/MyDrive/Colab Notebooks/SamSungTest2/TestData_withlabel/spamTest'

In [ ]:
dfTrain = makeDataFrame(dirTrainNotSpam, dirTrainSpam)
dfTrain

,Email,Label
0,Subject postings hi working phonetics project...,0
1,Subject precoling 92 symposium computational ...,0
2,Subject 3 378 korean l2 psycholinguistics ger...,0
3,Subject tone grammar summary finally lansing ...,0
4,Subject query icelandic anyone tell anything ...,0
...,...,...
362,Subject tinydrive usb 2 0 mobile 2 2 gb stora...,1
363,Subject payment confirmation p 9990835 begin ...,1
364,Subject viicodin via gra che ap jkoutsi lowes...,1
365,Subject 80 smart ink system www compusoft gr,1


In [ ]:
print(len(dfTrain['Label']))
print(dfTrain['Label'].sum())

367
174


In [ ]:
dfTest = makeDataFrame(dirTestNotSpam, dirTestSpam)
dfTest

,Email,Label
0,Subject query tagalog philippine informants n...,0
1,Subject humor communication call papers call ...,0
2,Subject 3 396 chomsky citations agree mark du...,0
3,Subject speaks languages vicki fromkin iyo1va...,0
4,Subject chomsky citations although faithful r...,0
...,...,...
88,Subject oxyyyyconttin script needeeed taaabbs...,1
89,Subject feel old go click help father pain go...,1
90,Subject penny stocks timing nomad internation...,1
91,Subject economize 55 recipes assists economis...,1


In [ ]:
print(dfTest['Label'].sum())

45


In [ ]:
# Tạo Dictionary (tu dien)

mySet = set()
for emailIndex in dfTrain['Email']:
  listwords = emailIndex.split()
  for wordIndex in listwords:
    mySet.add(wordIndex)

Dictionary = list(mySet)

In [ ]:
# Tạo weight_table

rows = len(dfTrain)
columns = len(Dictionary)
weight_table = np.zeros((rows, columns), dtype=int)
setx = set()

for i in range(len(dfTrain['Email'])):
  emailIndex = dfTrain['Email'][i]
  listwords = emailIndex.split()
  for wordIndex in listwords:
    if wordIndex in Dictionary:
      j = Dictionary.index(wordIndex)
      weight_table[i, j] = 1

In [ ]:
# Tính các giá trị cần thiết

y_train = dfTrain['Label']
y_test = dfTest['Label']
cntLabel = len(y_train)
cntLabelSpam = y_train.sum()
cntLabelNoSpam = cntLabel - cntLabelSpam

P_LabelSpam = cntLabelSpam / cntLabel
P_LabelNoSpam = cntLabelNoSpam / cntLabel

y_pred = pd.Series([0]*len(dfTest), name='y_pred')

In [ ]:
print("Số lượng nhãn: " + str(cntLabel))
print("Số lượng nhãn Spam: " + str(cntLabelSpam))
print("Sổ lượng nhãn NotSpam: " + str(cntLabelNoSpam))
print("Xác suất Nhãn Spam: " + str(P_LabelSpam))
print("Xác suất Nhãn NotSpam : " + str(P_LabelNoSpam))

Số lượng nhãn: 367
Số lượng nhãn Spam: 174
Sổ lượng nhãn NotSpam: 193
Xác suất Nhãn Spam: 0.47411444141689374
Xác suất Nhãn NotSpam : 0.5258855585831063


In [ ]:
# Tạo test_table
test_rows = len(dfTest)
test_columns = len(Dictionary)
test_table = np.zeros((test_rows, test_columns), dtype=int)

for i in range(len(dfTest['Email'])):
  emailIndex = dfTest['Email'][i]
  listwords = emailIndex.split()
  for wordIndex in listwords:
    if wordIndex in Dictionary:
      j = Dictionary.index(wordIndex)
      test_table[i, j] = 1


In [ ]:
# Dự đoán
# Tạo bảng Att

att_table = np.zeros((4, len(Dictionary)), dtype=np.double)

for j in range(columns):
  cnt_0_0 = 0
  cnt_1_0 = 0
  cnt_0_1 = 0
  cnt_1_1 = 0
  p_0_0 = 0
  p_1_0 = 0
  p_0_1 = 0
  p_1_1 = 0
  # cnt_x_1 = weight_table[:,j].sum
  # cnt_x_0 = len(weight_table) - cnt_x_1
  for i in range(rows):
    if weight_table[i, j] == 0 and y_train[i] == 0:
      cnt_0_0 += 1
    elif weight_table[i, j] == 1 and y_train[i] == 0:
      cnt_1_0 += 1
    elif weight_table[i, j] == 0 and y_train[i] == 1:
      cnt_0_1 += 1
    elif weight_table[i, j] == 1 and y_train[i] == 1:
      cnt_1_1 += 1
  # print(str(cnt_0_0) + " " + str(cnt_1_0) + " " + str(cnt_0_1) + " " + str(cnt_1_1))
  p_0_0 = np.double(cnt_0_0 + 1) / (cntLabelNoSpam + 1)
  p_1_0 = np.double(cnt_1_0 + 1) / (cntLabelNoSpam + 1)
  p_0_1 = np.double(cnt_0_1 + 1) / (cntLabelSpam + 1)
  p_1_1 = np.double(cnt_1_1 + 1) / (cntLabelSpam + 1)
  # print(str(p_0_0) + " " + str(p_1_0) + " " + str(p_0_1) + " " + str(p_1_1))
  # print(str(log(p_0_0)) + " " + str(log(p_1_0)) + " " + str(log(p_0_1)) + " " + str(log(p_1_1)))
  att_table[0][j] = log(p_0_0)
  att_table[1][j] = log(p_1_0)
  att_table[2][j] = log(p_0_1)
  att_table[3][j] = log(p_1_1)
  # print(str(j) + " " + str(cnt_0_1) + " " + str(p_0_1) + " " + str(log(p_0_1)) + " " + str(att_table[2][j]))

In [ ]:
print(att_table[0][0])
print(att_table[1][0])
print(att_table[2][0])
print(att_table[3][0])


-0.005167970158442561
-4.574710978503383
0.0
-5.1647859739235145


In [ ]:
for i in range(test_rows):
  cNB_spam = log(P_LabelSpam)
  cNB_no_spam = log(P_LabelNoSpam)
  for j in range(test_columns):
    if test_table[i][j] == 0:
      cNB_spam += att_table[2][j]
      cNB_no_spam += att_table[0][j]
    else:
      cNB_spam += att_table[3][j]
      cNB_no_spam += att_table[1][j]
  # y_pred[i] = int(1) if cNB_spam >= cNB_no_spam else 0
  if cNB_spam >= cNB_no_spam:
    y_pred[i] = 1

In [ ]:
print(accuracy_score(y_test, y_pred))

0.9032258064516129
